***GENERATED CODE FOR testjupyter PIPELINE***
***DON'T EDIT THIS CODE***

**CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM**

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


**TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA**

In [ ]:
import json
from pyspark.ml.feature import StandardScaler
from pyspark.sql.types import *
from pyspark.sql.functions import round, col
from pyspark.ml.feature import OneHotEncoderEstimator
from pyspark.sql.functions import col, round
from pyspark.sql.functions import col, udf
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import DoubleType, IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def vectorAssemblerTransform(df, param):

    dfReturn = df

    if (type(param) == str):
        outcol = param + "_vector"
        assembler = VectorAssembler(inputCols=[param], outputCol=outcol)
        dfReturn = assembler.transform(dfReturn)
        return dfReturn

    if (type(param) == list):
        vecAssembler = VectorAssembler(inputCols=param, outputCol="features")
        new_df = vecAssembler.transform(df)
        return new_df


def to_array(col):
    def to_array_(v):
        return v.toArray().tolist()
    return udf(to_array_, ArrayType(DoubleType()))(col)


def standardScalarTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: '0.0'})
    scalarFlags = transformationData["std_scalar"]
    if scalarFlags["mean_flag"]:
        stdflag = False
        meanflag = True
    elif scalarFlags["std_flag"]:
        stdflag = True
        meanflag = False

    outcol = feature + "_standardscalar"

    featureVector = feature + "_vector"
    dfReturn = vectorAssemblerTransform(dfReturn, feature)

    standardscale = StandardScaler(inputCol=featureVector, outputCol=outcol, withStd=stdflag,
                                   withMean=meanflag)
    scaledata = standardscale.fit(dfReturn).transform(dfReturn)
    dfReturn = scaledata.withColumn("final_col", to_array(scaledata[outcol]))\
        .select(scaledata.schema.names + [col("final_col")[0]])

    dfReturn = dfReturn.drop(outcol).drop(featureVector)\
        .withColumnRenamed("final_col[0]", outcol)
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[outcol], 2))

    return dfReturn


def OHETransform(df, params, transformationData={}):
    feature = params["feature"]
    dfReturn = df
    dfReturn = dfReturn.fillna({feature: ''})
    outcolS = feature + "_toint"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcolS, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcolS).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcolS, dfReturn[outcolS].cast(IntegerType()))
        dfReturn = changed_type_df

    outcol = feature + "_onehotencoder"
    encoder = OneHotEncoderEstimator(inputCols=[outcolS], outputCols=[
                                     outcol], handleInvalid="keep")
    model = encoder.fit(dfReturn)
    dfReturn = model.transform(dfReturn)
    dfReturn = dfReturn.drop(outcolS)
    dfReturn = dfReturn.withColumn("final_col", to_array(dfReturn[outcol])) \
        .select(dfReturn.schema.names + [col("final_col")[0]])
    dfReturn = dfReturn.drop(outcol).withColumnRenamed("final_col[0]", outcol)
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[outcol], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'State', 'transformation_label': 'String Indexer'}], 'feature': 'State', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '8', 'mean': '', 'stddev': '', 'min': 'HI', 'max': 'VA', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'State', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('State')
        transformationDF = standardScalarTransform(transformationDF, {'transformationsData': [{'feature_label': 'Account_Length', 'std_scalar': {'mean_flag': 'True', 'std_flag': 'False'}, 'transformation_label': 'Standard Scalar'}], 'feature': 'Account_Length', 'transformation': [
            {'transformation': 'Standard Scalar', 'selectedAsDefault': 1}], 'type': 'numeric', 'replaceby': 'mean', 'selected': 'True', 'stats': {'count': '8', 'mean': '111.75', 'stddev': '53.83', 'min': '33', 'max': '174', 'missing': '0'}}, {'feature_label': 'Account_Length', 'std_scalar': {'mean_flag': 'True', 'std_flag': 'False'}, 'transformation_label': 'Standard Scalar'})
        transformationDF = transformationDF.drop('Account_Length')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Phone', 'transformation_label': 'String Indexer'}], 'feature': 'Phone', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '8', 'mean': '', 'stddev': '', 'min': '329-9001', 'max': '410-7789', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'Phone', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Phone')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Intl_Plan', 'transformation_label': 'String Indexer'}], 'feature': 'Intl_Plan', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '8', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'Intl_Plan', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Intl_Plan')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'VMail_Plan', 'transformation_label': 'String Indexer'}, {'feature_label': 'VMail_Plan', 'transformation_label': 'One Hot Encoding'}], 'feature': 'VMail_Plan', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '8', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}, {'transformation': 'One Hot Encoding', 'selectedAsDefault': 0}]}, {'feature_label': 'VMail_Plan', 'transformation_label': 'String Indexer'})
        transformationDF = OHETransform(transformationDF, {'transformationsData': [{'feature_label': 'VMail_Plan', 'transformation_label': 'String Indexer'}, {'feature_label': 'VMail_Plan', 'transformation_label': 'One Hot Encoding'}], 'feature': 'VMail_Plan', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '8', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}, {'transformation': 'One Hot Encoding', 'selectedAsDefault': 0}]}, {'feature_label': 'VMail_Plan', 'transformation_label': 'One Hot Encoding'})
        transformationDF = transformationDF.drop('VMail_Plan')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'cluster_labels', 'transformation_label': 'String Indexer'}], 'feature': 'cluster_labels', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '8', 'mean': '', 'stddev': '', 'min': 'day_callers', 'max': 'vmailers', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'cluster_labels', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('cluster_labels')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


**AUTOML FUNCTIONS**

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = df.drop(label, axis=1).values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': sparkDF.columns}
    return data


**READING DATAFRAME**

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

%run testjupyterHooks.ipynb
try:
	sourcePreExecutionHook()

	testjupyterdbfs = DBFSConnector.fetch(spark, "{'url': '/Demo/PredictiveChurnTest.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line'}")

	sourcePostExecutionHook(testjupyterdbfs)

except Exception as ex: 
	logging.error(ex)


**TRANSFORMING DATAFRAME**

In [ ]:
%run testjupyterHooks.ipynb
try:
	transformationPreExecutionHook()

	testjupyterautofe = TransformationMain.run(testjupyterdbfs,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "State", "transformation_label": "String Indexer"}], "feature": "State", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "8", "mean": "", "stddev": "", "min": "HI", "max": "VA", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "Account_Length", "std_scalar": {"mean_flag": "True", "std_flag": "False"}, "transformation_label": "Standard Scalar"}], "feature": "Account_Length", "transformation": [{"transformation": "Standard Scalar", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8", "mean": "111.75", "stddev": "53.83", "min": "33", "max": "174", "missing": "0"}}, {"transformationsData": [{}], "feature": "Area_Code", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8", "mean": "437.0", "stddev": "45.16", "min": "408", "max": "510", "missing": "0"}}, {"transformationsData": [{"feature_label": "Phone", "transformation_label": "String Indexer"}], "feature": "Phone", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "8", "mean": "", "stddev": "", "min": "329-9001", "max": "410-7789", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "Intl_Plan", "transformation_label": "String Indexer"}], "feature": "Intl_Plan", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "8", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "VMail_Plan", "transformation_label": "String Indexer"}, {"feature_label": "VMail_Plan", "transformation_label": "One Hot Encoding"}], "feature": "VMail_Plan", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "8", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}, {"transformation": "One Hot Encoding", "selectedAsDefault": 0}]}, {"transformationsData": [{}], "feature": "VMail_Message", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8", "mean": "3.62", "stddev": "10.25", "min": "0", "max": "29", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Day_Mins", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8", "mean": "142.51", "stddev": "35.99", "min": "81.1", "max": "187.7", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{}], "feature": "Day_Calls", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8", "mean": "97.12", "stddev": "17.69", "min": "76", "max": "127", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Day_Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8", "mean": "24.23", "stddev": "6.12", "min": "13.79", "max": "31.91", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{}], "feature": "Eve_Mins", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8", "mean": "197.29", "stddev": "90.75", "min": "103.1", "max": "351.6", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{}], "feature": "Eve_Calls", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8", "mean": "95.75", "stddev": "26.31", "min": "71", "max": "148", "missing": "0"}}, {"transformationsData": [{}], "feature": "Eve_Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8", "mean": "16.77", "stddev": "7.71", "min": "8.76", "max": "29.89", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{}], "feature": "Night_Mins", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8", "mean": "213.79", "stddev": "43.13", "min": "141.1", "max": "279.2", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{}], "feature": "Night_Calls", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8", "mean": "100.88", "stddev": "16.57", "min": "79", "max": "128", "missing": "0"}}, {"transformationsData": [{}], "feature": "Night_Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8", "mean": "9.62", "stddev": "1.94", "min": "6.35", "max": "12.56", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{}], "feature": "Intl_Mins", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8", "mean": "10.86", "stddev": "2.76", "min": "7.1", "max": "15.5", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{}], "feature": "total_Mins", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8", "mean": "564.45", "stddev": "113.42", "min": "386.0", "max": "760.6", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{}], "feature": "Intl_Calls", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8", "mean": "4.5", "stddev": "3.16", "min": "1", "max": "11", "missing": "0"}}, {"transformationsData": [{}], "feature": "Intl_Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8", "mean": "2.93", "stddev": "0.74", "min": "1.92", "max": "4.19", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{}], "feature": "Total_Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8", "mean": "53.55", "stddev": "10.11", "min": "40.19", "max": "73.32", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{}], "feature": "CustServ_Calls", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8", "mean": "1.25", "stddev": "1.49", "min": "0", "max": "4", "missing": "0"}}, {"transformationsData": [{}], "feature": "Churn", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8", "mean": "0.12", "stddev": "0.35", "min": "0", "max": "1", "missing": "0"}}, {"transformationsData": [{"feature_label": "cluster_labels", "transformation_label": "String Indexer"}], "feature": "cluster_labels", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "8", "mean": "", "stddev": "", "min": "day_callers", "max": "vmailers", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"feature": "State_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{}], "type": "real", "selected": "True", "stats": {"count": "8", "mean": "3.5", "stddev": "2.45", "min": "0.0", "max": "7.0", "missing": "0"}}, {"feature": "Phone_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{}], "type": "real", "selected": "True", "stats": {"count": "8", "mean": "3.5", "stddev": "2.45", "min": "0.0", "max": "7.0", "missing": "0"}}, {"feature": "Intl_Plan_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{}], "type": "numeric", "selected": "True", "stats": {"count": "8", "mean": "0.12", "stddev": "0.35", "min": "0", "max": "1", "missing": "0"}}, {"feature": "VMail_Plan_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{}], "type": "numeric", "selected": "True", "stats": {"count": "8", "mean": "0.12", "stddev": "0.35", "min": "0", "max": "1", "missing": "0"}}, {"feature": "cluster_labels_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{}], "type": "real", "selected": "True", "stats": {"count": "8", "mean": "1.38", "stddev": "1.51", "min": "0.0", "max": "4.0", "missing": "0"}}, {"feature": "Account_Length_standardscalar_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "selected": "True", "transformationsData": [{}], "type": "real", "stats": {"count": "8", "mean": "0.0", "stddev": "53.83", "min": "-78.75", "max": "62.25", "missing": "0"}}, {"feature": "VMail_Plan_onehotencoder", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "selected": "True", "transformationsData": [{}], "type": "real", "stats": {"count": "8", "mean": "0.88", "stddev": "0.35", "min": "0.0", "max": "1.0", "missing": "0"}}]}))

	transformationPostExecutionHook(testjupyterautofe)

except Exception as ex: 
	logging.error(ex)


**TRAIN MODEL**

In [ ]:
%run testjupyterHooks.ipynb
try:
	mlPreExecutionHook()

	dataAutoML=functionClassification(testjupyterautofe, ["Area_Code", "VMail_Message", "Day_Mins", "Day_Calls", "Day_Charge", "Eve_Mins", "Eve_Calls", "Eve_Charge", "Night_Mins", "Night_Calls", "Night_Charge", "Intl_Mins", "total_Mins", "Intl_Calls", "Intl_Charge", "Total_Charge", "CustServ_Calls", "State_stringindexer", "Phone_stringindexer", "Intl_Plan_stringindexer", "VMail_Plan_stringindexer", "cluster_labels_stringindexer", "Account_Length_standardscalar", "VMail_Plan_onehotencoder"], "Churn")

	mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


**PREDICT ON TRAINED MODEL**

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

